In [10]:
import tensorflow as tf
from tensorflow import keras

In [11]:
# from keras.layers import Dropout
# from keras import regularizers, optimizers
# from keras.layers import Input, Conv1D, Dense, Flatten, Activation, UpSampling1D, MaxPooling1D, ZeroPadding1D, TimeDistributed
# from keras.models import Model, load_model
# from keras.layers.core import Reshape
#
# from keras.models import Sequential
# from keras.layers import LSTM
# from keras.layers import Dense
# from keras.layers import RepeatVector
# from keras.layers import TimeDistributed
# from keras.utils import plot_model
#
# from keras.layers import Input, LSTM, RepeatVector
# from keras.models import Model

In [12]:
class Hyperparameters():
    r = 5

class Model(object):
    def __init__(self, params):
        # model parameters
        self.r = params.r

    def mogrify(self, x_init, h_0, q, r):
        x = x_init
        h_prev = h_0
        for i in range(1, self.r+1):
            if i % 2 != 0:
                x = 2 * tf.sigmoid(tf.matmul(q[i-1], h_prev)) * x
                # print("odd %s, %s" % (i, tf.shape(x)))
            else:
                h_prev = 2 * tf.sigmoid(tf.matmul(r[i-1], x)) * h_prev
                # print("even %s, %s" % (i, tf.shape(h_prev)))

        return x, h_prev

    def __call__(self, x):
        return

In [14]:
# add more q och r, they seem to be different matrices each

h_0 = tf.random.normal([20,10]) # n x a
x = tf.random.normal([100, 10]) # d x a

params = Hyperparameters()
model = Model(params)

q = [tf.random.normal([100, 20]) for _ in range(model.r)] # m x n, where m == d
r = [tf.random.normal([20, 100]) for _ in range(model.r)] # n x m, where m == d, it's just transposed Q
model.mogrify(x, h_0, q, r)

(<tf.Tensor: shape=(100, 10), dtype=float32, numpy=
 array([[ 5.92775941e-01,  1.02219519e-06,  7.52691460e+00,
          5.05843345e-05, -4.00337428e-01,  6.41464889e-01,
         -6.83444366e-02, -3.49220682e-05,  3.05544496e-01,
          4.66716677e-01],
        [-1.46195933e-03,  2.34880488e-07,  8.06811273e-01,
          5.74071333e-02,  3.65012698e-02,  1.41610729e-03,
          2.48389053e+00,  1.19882655e+00,  1.07587093e-05,
         -3.96958749e-05],
        [ 1.12133194e-02, -1.15813594e-03, -1.25669865e-14,
         -1.84942055e-02,  4.04450810e-04,  2.71275803e-05,
          2.47465796e-03, -9.71779102e-09,  5.74422553e-02,
         -9.29677486e-01],
        [-1.78453517e+00, -2.26179272e-01,  6.32319748e-01,
          1.45954033e-03,  2.57790666e-02,  1.47480657e-02,
          2.36324687e-03, -4.93072905e-02, -6.41271162e-13,
          1.49959655e-04],
        [ 4.09280968e+00, -3.25707644e-01,  5.68715953e-18,
          4.80921294e-08, -8.55796039e-02,  5.97445011e-01,
